In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

from rb_equivariant_cnn import RB3D_Conv
from rb_equivariant_gcnn import RB3D_G_Conv

2024-07-31 23:18:34.603594: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Setup

In [2]:
RB_CHANNELS = 4
HORIZONTAL_SIZE = 64
HEIGHT = 32

BATCH_SIZE = 64

# Convolution

In [5]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, name='Conv1', padding='SAME'),
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, name='Conv2', padding='VALID'),
        ])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (RB3D_Conv)               │ (64, 64, 64, 28, 4)    │        20,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 64, 64, 24, 4)    │        17,376 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,648 (147.06 KB)

 Trainable params: 37,648 (147.06 KB)

 Non-trainable params: 0 (0.00 B)

# D4 Group Equivariant Convolution

In [4]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            RB3D_G_Conv('Z2', 'D4', v_ksize=5, h_ksize=3, channels=RB_CHANNELS, name='G_Conv1'),
            RB3D_G_Conv('D4', 'D4', v_ksize=5, h_ksize=3, channels=RB_CHANNELS, name='G_Conv2'),
        ])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ G_Conv1 (RB3D_G_Conv)           │ (64, 62, 62, 8, 28, 4) │        20,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ G_Conv2 (RB3D_G_Conv)           │ (64, 60, 60, 8, 24, 4) │       138,336 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 158,608 (619.56 KB)

 Trainable params: 158,608 (619.56 KB)

 Non-trainable params: 0 (0.00 B)